In [1]:


import csv
from pathlib import Path

import cv2
import imageio
import imageio_ffmpeg
import numpy as np
import onnxruntime as ort
import torch
import os
import torchvision.transforms as T
from PIL import Image
from sparrow_datums import BoxTracking
from tqdm import tqdm
from torch.utils.data import DataLoader
from speed_trapv3.detection.dataset import RetinaNetDataset
from speed_trapv3.utils import Holdout
import matplotlib.pyplot as plt
from speed_trapv3.detection.model import RetinaNet
from speed_trapv3.detection.config import Config
import matplotlib.pyplot as plt

data = RetinaNetDataset(Holdout.DEV)
dataloader = DataLoader(data, batch_size=1, shuffle=False)
model = RetinaNet().eval().cuda()
model.load(Config.trained_model_path)
count = 0
for batch in iter(dataloader):
    x = batch['image']
    x = x.cuda()
    result = model(x)
    bbxes = result[0]["boxes"].detach().cpu().numpy()
    # keypoints = batch['keypoints'].detach().cpu().numpy()[0]
    transform = T.ToPILImage()
    img = np.array(transform(x[0]))
    for bbox in bbxes:
        x1, y1, x2, y2 = bbox.astype(int)
        img = cv2.rectangle(img, (x1, y1), (x2, y2), (0, 255, 0), thickness=4)
    # img = cv2.circle(img, (int(keypoints[0][0]), int(keypoints[0][1])), radius=10, color=(0, 0, 255), thickness=-1) #Blue: Backtire
    # img = cv2.circle(img, (int(keypoints[1][0]), int(keypoints[1][1])), radius=10, color=(255, 0, 0), thickness=-1) #Red Front tire
    save_path = '/code/data/datasets/temp_imgs'
    count += 1
    filename = str(count) + ".jpg"
    cv2.imwrite(os.path.join(save_path, filename), img)


/usr/local/lib/python3.9/dist-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/usr/lib/python3/dist-packages/requests/__init__.py:89: RequestsDependencyWarning: urllib3 (1.26.12) or chardet (3.0.4) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({}) doesn't match a supported "
/usr/local/lib/python3.9/dist-packages/torch/functional.py:445: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at  ../aten/src/ATen/native/TensorShape.cpp:2157.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


: 

In [ ]:
result